In [1]:
import csv
from time import time
from datetime import timedelta

import inflect

def create_data(num):
    p = inflect.engine()

    filename=["first", "second", "third", "fourth", "fifth","sixth", "sevent", "eight", "ninth", "tenth"]
    filename = filename[:num]

    for index, file in enumerate(filename):
        with open(f"{file}.csv", "w", newline="") as csvfile:
            csvwriter = csv.writer(csvfile)
            columns = 1_0
            rows = 1_0
            st = time()
            csvwriter.writerows([[file]+[f"{p.number_to_words(r+1)}_{p.number_to_words(c+1)}" for c in range(columns)] for r in range(rows)])
            et = time()
            print(f"[HH:MM:SS]{timedelta(seconds=et-st)}")
            print(f"{(index+1)*100/len(filename)}%") # percent complete

In [3]:
create_data(2)

[HH:MM:SS]0:01:01.755295
50.0%
[HH:MM:SS]0:01:02.311080
100.0%


In [3]:
%%file reducer_test.py
import sys
from glob import glob
from time import time
from datetime import timedelta
from functools import reduce, partial

from memory_profiler import profile


@profile
def reducer(num=None):
    st = time()

    received_csv_files = [file for file in glob("*.csv") if file != "cumulative.csv"]
    if num == None or 0 > num > len(received_csv_files):
        num = len(received_csv_files)
    received_csv_files = received_csv_files[:num]
    print(received_csv_files)

    open_csv_files = map(partial(open, mode="rb"), received_csv_files)

    with open("cumulative.csv", "wb") as outfile:

        def combine_files(out_file, in_file):
            #             if out_file.tell() == 0:
            #                 out_file.writelines(in_file)
            #             else:
            #                 next(in_file)
            #                 out_file.writelines(in_file)
            #             in_file.close()
            #             return out_file
            for line in in_file:
                if out_file.tell() == 0:
                    out_file.write(line)
                elif in_file.tell() == 0:
                    next(in_file)
                    out_file.write(line)
                else:
                    out_file.write(line)
            in_file.close()
            return out_file

        reduce(combine_files, open_csv_files, outfile)

    et = time()
    print(f"[HH:MM:SS]{timedelta(seconds=et-st)}")


if __name__ == "__main__":
    reducer(int(sys.argv[1]))


Overwriting reducer_test.py


In [5]:
%load_ext memory_profiler

In [16]:
%memit reducer(2)

ERROR: Could not find file /tmp/ipykernel_5571/440761055.py
['first.csv', 'fourth.csv']
[HH:MM:SS]0:00:00.153600
peak memory: 92.79 MiB, increment: 0.01 MiB


In [2]:
create_data(10)

[HH:MM:SS]0:00:00.006058
10.0%
[HH:MM:SS]0:00:00.005821
20.0%
[HH:MM:SS]0:00:00.005749
30.0%
[HH:MM:SS]0:00:00.005464
40.0%
[HH:MM:SS]0:00:00.005435
50.0%
[HH:MM:SS]0:00:00.005318
60.0%
[HH:MM:SS]0:00:00.005304
70.0%
[HH:MM:SS]0:00:00.005275
80.0%
[HH:MM:SS]0:00:00.005502
90.0%
[HH:MM:SS]0:00:00.005400
100.0%


In [17]:
%memit reducer(10)

ERROR: Could not find file /tmp/ipykernel_5571/440761055.py
['first.csv', 'fourth.csv', 'sixth.csv', 'fifth.csv', 'third.csv', 'sevent.csv', 'tenth.csv', 'second.csv', 'ninth.csv', 'eight.csv']
[HH:MM:SS]0:00:00.783398
peak memory: 92.79 MiB, increment: 0.00 MiB
